In [ ]:
# NASDAQ Stock Prediction - Google Colab
# =======================================

# %% [markdown]
# # 🚀 Predicción de Acciones NASDAQ en Google Colab
# 
# Este notebook ejecuta TODO el pipeline en Colab con GPU gratis
# 
# **Duración estimada:** 2-3 horas

# %% [markdown]
# ## ⚙️ PASO 1: Configurar GPU

# %%
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU disponible:", tf.config.list_physical_devices('GPU'))

# Verificar memoria disponible
!cat /proc/meminfo | grep MemTotal

# %% [markdown]
# ## 📂 PASO 2: Montar Google Drive y descomprimir proyecto

# %%
from google.colab import drive
import os
import zipfile

# Montar Drive
drive.mount('/content/drive', force_remount=True)

# Verificar que el ZIP existe
zip_path = '/content/drive/MyDrive/NASDAQ_Project/proyecto_nasdaq.zip'

if os.path.exists(zip_path):
    print(f"✅ ZIP encontrado: {zip_path}")
    
    # Descomprimir en /content/ (más rápido que Drive)
    print("📦 Descomprimiendo proyecto...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    
    print("✅ Proyecto descomprimido")
else:
    print(f"❌ ERROR: No se encontró el archivo en {zip_path}")
    print("\n🔧 Solución:")
    print("1. Sube 'proyecto_nasdaq.zip' a tu Google Drive")
    print("2. Colócalo en: MyDrive/NASDAQ_Project/")
    print("3. O cambia la ruta 'zip_path' arriba")

# %%
# Ir a la carpeta del proyecto
%cd /content/NASDAQ_Stock_Prediction_Project

# Verificar estructura
print("\n📁 Estructura del proyecto:")
!ls -la

# Verificar que stocks_full.csv existe
if os.path.exists('data/processed/stocks_full.csv'):
    import pandas as pd
    df = pd.read_csv('data/processed/stocks_full.csv', nrows=5)
    print("\n✅ stocks_full.csv encontrado")
    print(f"   Columnas: {df.columns.tolist()}")
else:
    print("❌ ERROR: data/processed/stocks_full.csv NO existe")
    print("   Debes incluirlo en el ZIP")

# %% [markdown]
# ## 📦 PASO 3: Instalar dependencias

# %%
# Instalar todas las dependencias
!pip install -q pandas numpy scikit-learn matplotlib seaborn plotly pyyaml tqdm ta

print("✅ Dependencias instaladas")

# Verificar versiones clave
import pandas as pd
import numpy as np
print(f"\nVersiones:")
print(f"  Pandas: {pd.__version__}")
print(f"  NumPy: {np.__version__}")
print(f"  TensorFlow: {tf.__version__}")

# %% [markdown]
# ## 🔧 PASO 4: Ajustar configuración para Colab

# %%
import yaml

# Leer config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("📋 Configuración actual:")
print(f"  Lookback: {config['features']['lookback_window']}")
print(f"  Epochs LSTM: {config['models']['lstm']['epochs']}")
print(f"  Batch size: {config['models']['lstm']['batch_size']}")

# AJUSTAR PARA COLAB (opcional - para que sea más rápido)
# Descomenta si quieres prueba rápida:

# config['features']['lookback_window'] = 30  # Reducir RAM
# config['models']['lstm']['epochs'] = 20     # Menos epochs
# config['models']['lstm']['batch_size'] = 64 # Batch más grande
# 
# with open('config.yaml', 'w') as f:
#     yaml.dump(config, f)
# 
# print("\n✅ Config ajustado para Colab")

# %% [markdown]
# ## 📊 PASO 5: Exploración rápida de datos

# %%
from src.utils import load_config, create_directories
from src.data_loader import StockDataLoader

config = load_config('config.yaml')
create_directories(config)

loader = StockDataLoader(config)
df = loader.load_full_data()

print(f"\n📊 Dataset filtrado:")
print(f"  Filas: {len(df):,}")
print(f"  Tickers: {df['ticker'].nunique()}")
print(f"  Rango: {df['Date'].min().date()} → {df['Date'].max().date()}")
print(f"  RAM usada: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# Top 10 tickers
print(f"\n🔝 Top 10 tickers por registros:")
print(df['ticker'].value_counts().head(10))

# %% [markdown]
# ## 🗂️ PASO 6: Generar Splits (Train/Val/Test)

# %%
import os

# Verificar si ya existen
if os.path.exists('data/processed/train.csv'):
    print("✅ Splits ya existen, saltando generación...")
else:
    print("📊 Generando splits...")
    train, val, test = loader.split_data(df)
    loader.save_splits(train, val, test)
    print("✅ Splits generados")

# %% [markdown]
# ## 🏋️ PASO 7: ENTRENAR MODELOS
# 
# **⚠️ IMPORTANTE:** 
# - Esto tardará 1-3 horas dependiendo del modelo
# - Colab Free puede desconectarse después de ~12 horas
# - Recomiendo entrenar de a un modelo por sesión

# %% [markdown]
# ### Opción A: Entrenar SOLO LSTM (30-60 min)

# %%
from src.train import TrainingPipeline
import warnings
warnings.filterwarnings('ignore')

print("🏋️ Entrenando LSTM...")
print("⏱️ Tiempo estimado: 30-60 minutos en GPU")
print("\n" + "="*70)

pipeline = TrainingPipeline('config.yaml')

try:
    trained_models = pipeline.run(models=['lstm'])
    print("\n✅ LSTM entrenado exitosamente")
except Exception as e:
    print(f"\n❌ ERROR: {e}")
    print("\n🔧 Si el error es de memoria:")
    print("   1. Reduce lookback_window a 30 en config.yaml")
    print("   2. O filtra menos tickers (selected_tickers)")

# %% [markdown]
# ### Opción B: Entrenar LSTM + GRU (1-2 horas)

# %%
# ⚠️ Descomenta para entrenar 2 modelos
# pipeline = TrainingPipeline('config.yaml')
# trained_models = pipeline.run(models=['lstm', 'gru'])

# %% [markdown]
# ### Opción C: Entrenar TODOS (2-4 horas)

# %%
# ⚠️ Solo para Colab Pro o si tienes tiempo
# pipeline = TrainingPipeline('config.yaml')
# trained_models = pipeline.run(models=['lstm', 'gru', 'tcn', 'tft'])

# %% [markdown]
# ## 📊 PASO 8: Visualizar Historial de Entrenamiento

# %%
import matplotlib.pyplot as plt
import json

# Cargar historial del modelo entrenado
with open('results/models/lstm_history.json', 'r') as f:
    history = json.load(f)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history['loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_title('LSTM - Loss durante entrenamiento', fontweight='bold', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (MSE)')
ax1.legend()
ax1.grid(alpha=0.3)

# MAE
ax2.plot(history['mae'], label='Train MAE', linewidth=2)
ax2.plot(history['val_mae'], label='Val MAE', linewidth=2)
ax2.set_title('LSTM - MAE durante entrenamiento', fontweight='bold', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Mejor val_loss: {min(history['val_loss']):.6f}")
print(f"   Alcanzado en epoch: {history['val_loss'].index(min(history['val_loss'])) + 1}")

# %% [markdown]
# ## 🧪 PASO 9: Evaluar en Test Set

# %%
from src.evaluate import ModelEvaluator
import pickle

print("📊 Preparando evaluación...")

# Cargar scalers
with open('results/models/feature_scaler.pkl', 'rb') as f:
    feature_scaler = pickle.load(f)
with open('results/models/target_scaler.pkl', 'rb') as f:
    target_scaler = pickle.load(f)

# Cargar metadata
with open('results/models/metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"✅ Metadata cargado")
print(f"   Features: {len(metadata['feature_cols'])}")
print(f"   Test samples: {metadata['test_samples']:,}")

# %% [markdown]
# **⚠️ NOTA:** Para evaluación completa, necesitas ejecutar el pipeline
# completo que genera X_test, y_test. Ver notebook 03_train_and_evaluate.ipynb

# %%
# Evaluación simplificada - cargar modelo y verificar
from src.models.lstm import LSTMModel

input_shape = tuple(metadata['input_shape'])
lstm_model = LSTMModel(config, input_shape)
lstm_model.load_weights('results/models/lstm_best.h5')

print("✅ Modelo LSTM cargado correctamente")
lstm_model.summary()

# %% [markdown]
# ## 💾 PASO 10: Descargar Resultados

# %%
# Comprimir resultados
!zip -r results_nasdaq.zip results/

print("\n📦 Resultados comprimidos en: results_nasdaq.zip")
print("   Tamaño:")
!du -h results_nasdaq.zip

# Descargar
from google.colab import files
files.download('results_nasdaq.zip')

print("\n✅ ¡Descarga iniciada!")

# %% [markdown]
# ## 🎉 ¡COMPLETADO!
# 
# ### 📋 Resumen de archivos generados:
# - **Modelos**: `results/models/*.h5`
# - **Gráficos**: `results/plots/*.png`
# - **Tablas**: `results/tables/*.csv`
# - **Scalers**: `results/models/*_scaler.pkl`
# 
# ### 🔄 Próximos pasos:
# 1. Descarga `results_nasdaq.zip`
# 2. Descomprímelo en tu proyecto local
# 3. Analiza resultados en tu computadora
# 4. Repite con otros modelos (GRU, TCN, TFT)
# 
# ### 💡 Tips:
# - Guarda este notebook en tu Drive
# - Los modelos entrenados se pueden reutilizar
# - Para entrenar otros modelos, solo cambia `models=['gru']`

# %% [markdown]
# ---
# **Creado para el proyecto NASDAQ Stock Prediction**  
# Si hay errores, revisa la sección de configuración y ajusta según tu RAM/GPU